In [19]:
import requests
import json
import backoff
import random
import logging

In [2]:
url = 'https://economia.awesomeapi.com.br/json/last/USD-BRL'
ret = requests.get(url)

In [3]:
if ret:
    print(ret)
else:
    print('Falhou')

<Response [200]>


In [4]:
dolar = json.loads(ret.text)['USDBRL']

In [5]:
dolar

{'code': 'USD',
 'codein': 'BRL',
 'name': 'Dólar Americano/Real Brasileiro',
 'high': '5.1389',
 'low': '5.0933',
 'varBid': '0.0238',
 'pctChange': '0.46',
 'bid': '5.1342',
 'ask': '5.1367',
 'timestamp': '1660055251',
 'create_date': '2022-08-09 11:27:31'}

In [6]:
print(f"20 dólares hoje custam {float(dolar['bid']) * 20} reais")

20 dólares hoje custam 102.684 reais


In [7]:
def cotacao(valor, moeda):
    url = f'https://economia.awesomeapi.com.br/json/last/{moeda}'
    # url = 'https://economia.awesomeapi.com.br/json/last/{}'.format(moeda)
    ret = requests.get(url)
    dolar = json.loads(ret.text)[moeda.replace('-', '')]
    print(f"{valor} {moeda[:3]} hoje custam {float(dolar['bid']) * valor} {moeda[-3:]}")
    

In [8]:
cotacao(20, 'USD-BRL')

20 USD hoje custam 102.684 BRL


In [9]:
cotacao(20, 'JPY-BRL')

20 JPY hoje custam 0.7606 BRL


In [10]:
try:
    cotacao(20, Thiago)
except Exception as e:
    print(e)
else:
    print('ok')

name 'Thiago' is not defined


In [11]:
lst_money = [
        "USD-BRL",
        "EUR-BRL",
        "BTC-BRL",
        "JPY-BRL",
        "RPL-BRL"
    ]

In [12]:
def error_check(func):
    def inner_func(*args, **kargs):
        try:
            func(*args, **kargs)
        except:
            print(f"{func.__name__} falhou")
    return inner_func

In [13]:
@error_check
def multi_moeda(valor, moeda):
    url = f'https://economia.awesomeapi.com.br/json/last/{moeda}'
    ret = requests.get(url)
    dolar = json.loads(ret.text)[moeda.replace('-', '')]
    print(f"{valor} {moeda[:3]} hoje custam {float(dolar['bid']) * valor} {moeda[-3:]}")
    

In [15]:
multi_moeda(20,  "USD-BRL")
multi_moeda(20,  "EUR-BRL")
multi_moeda(20,  "BTC-BRL")
multi_moeda(20,  "RPL-BRL")
multi_moeda(20,  "JPY-BRL")


20 USD hoje custam 102.684 BRL
20 EUR hoje custam 105.062 BRL
20 BTC hoje custam 2371.38 BRL
multi_moeda falhou
20 JPY hoje custam 0.7606 BRL


In [20]:
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=10)
def test_func(*args, **kargs):
    rnd = random.random()
    print(f"""
            RND: {rnd}
            args: {args if args else 'sem args'}
            kargs: {kargs if kargs else 'sem kargs'}
        """)
    if rnd < .2:
        raise ConnectionAbortedError('Conexão foi finalizada')
    elif rnd < .4:
        raise ConnectionRefusedError('Conexão foi recusada')
    elif rnd < .6:
        raise TimeoutError('Tempo de espera excedido')
    else:
        return "OK!"

In [21]:
test_func()


            RND: 0.8768112395999444
            args: sem args
            kargs: sem kargs
        


'OK!'

In [24]:
test_func(42, 51, nome='Thiago')


            RND: 0.321600552529148
            args: (42, 51)
            kargs: {'nome': 'Thiago'}
        

            RND: 0.8442765338546993
            args: (42, 51)
            kargs: {'nome': 'Thiago'}
        


'OK!'

In [27]:
log = logging.getLogger()
log.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler()
ch.setFormatter(formatter)
log.addHandler(ch)

In [28]:
@backoff.on_exception(backoff.expo, (ConnectionAbortedError, ConnectionRefusedError, TimeoutError), max_tries=10)
def test_func(*args, **kargs):
    rnd = random.random()
    log.debug(f" RND:{rnd}")
    log.info(f"args: {args if args else 'sem args'}")
    log.info(f"kargs: {kargs if kargs else 'sem kargs'}")
    if rnd < .2:
        log.error('Conexão foi finalizada')
        raise ConnectionAbortedError('Conexão foi finalizada')
    elif rnd < .4:
        log.error('Conexão foi recusada')
        raise ConnectionRefusedError('Conexão foi recusada')
    elif rnd < .6:
        log.error('Tempo de espera excedido')
        raise TimeoutError('Tempo de espera excedido')
    else:
        return "OK!"

In [29]:
test_func()

2022-08-09 12:23:11,585 - root - DEBUG -  RND:0.59454127776428
2022-08-09 12:23:11,585 - root - INFO - args: sem args
2022-08-09 12:23:11,586 - root - INFO - kargs: sem kargs
2022-08-09 12:23:11,586 - root - ERROR - Tempo de espera excedido
2022-08-09 12:23:11,586 - backoff - INFO - Backing off test_func(...) for 0.6s (TimeoutError: Tempo de espera excedido)
2022-08-09 12:23:12,183 - root - DEBUG -  RND:0.9870898355238926
2022-08-09 12:23:12,183 - root - INFO - args: sem args
2022-08-09 12:23:12,184 - root - INFO - kargs: sem kargs


'OK!'